In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
import ApproxFun as AF
import DifferentialEquations as DE

# Harmonic Balance using multiple harmonics

## **Try out different methods to do the Symbolic computations on higher harmonics**

Using Python's famous SymPy

In [ ]:
@time begin
    x = sp.symbols("x")
    f = sp.cos(40x)^4
    
    fs = sp.sympy.fourier_series(f, (x, -sp.PI, sp.PI))
end
println(fs.truncate(n=10))

Using Julia's ApproxFun.jl

In [ ]:
@time begin
    # Define on CosSpace (cosine series only)
    f = AF.Fun(x -> cos(5x)^4, AF.CosSpace())
    
    coeffs = AF.coefficients(f)
end

println("Cosine Series:")
for (k, c) in enumerate(coeffs)
        if abs(c) > 1e-10
            println("$(c) * cos($((k-1))x)")
        end
end

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0.0;
omega = 40;
gamma3 = 60.0;
g0 = 9.80665; # m / s^2
height = 5; # m

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
N = 1000;
harmonics = 5; # number of harmonics
order = 2;
stepx = (xright-xleft)/N;

In [ ]:
# Define symbolics
Model.@parameters x, t;
u = 0
j = 1
vars = []

bcs = []

for i in 1:(2*harmonics)
    # Create variable A(..), B(..), ...
    c = Char('A' + i - 1)
    v = eval(:(Model.@variables $(Symbol(c))(..)))
    

 
    # Call the function with argument(s) when using it
    if isodd(i)
        u += v[1](x) * sin(j * omega*t)  # v[1](t) instead of v
    else
        u += v[1](x) * cos(j * omega*t)
        j+=1
    end
    
    push!(vars, v[1](x))  # v is a vector, v[1] is the function
    push!(bcs, v[1](xleft) ~ 0)
    push!(bcs, v[1](xright) ~ 0)
    
end

F = 100 * exp(-10*x^2)*sin(omega*t)
Dx = Model.Differential(x);
Dt = Model.Differential(t);
y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;
y_exp = Symb.expand(Model.expand_derivatives(y));
symbolics_list = Symb.arguments(y_exp, +);

The assumed ansatz looks the following

In [ ]:
u

In [ ]:
t_sympy = sp.symbols("t")
x_sympy = sp.symbols("x")
Symb.@variables x t;
finished_terms = []  # Initialize empty list

"""
The idea for this new implementation is to use the form of the PDE with the ansatz substituted in and extract each term (separate wrt addition/subtraction)

Each term is then converted from the Symbolics.jl form to the SymPy form and the part dependent on t is extracted for the Fourier series. The expanded
Fourier Series term is divided by the SymPy non-simplified term. Convert back to Symbolics.jl this form and by multiplying it with the original "term" the 
trigonometric term to the power (e.g. cos(x)^3) is cancelled out.
"""


for (i, term) in enumerate(symbolics_list)
    # Transform term by term into sympy form
    term_sympy = Symb.symbolics_to_sympy(term)
    current_term = term_sympy.as_independent(t_sympy)[2] # extract term sinusoidal term dependent on t
    
    fs = sp.sympy.fourier_series(current_term, (t_sympy, -sp.PI, sp.PI)) # simplify using Sympy's Fourier Series
    finished_sympy_term = fs.truncate() / current_term 
    finished_symb_term = Symb.sympy_to_symbolics(finished_sympy_term, [t])*symbolics_list[i]
    push!(finished_terms, finished_symb_term)
    # println(finished_symb_term)

end

In [ ]:
final_expression = Symb.simplify(sum(finished_terms));

eqs = []

for i in 1:harmonics
    sin_coef = Symb.coeff(final_expression, sin(i*omega*t));
    cos_coef = Symb.coeff(final_expression, cos(i*omega*t));
    push!(eqs, sin_coef ~ 0)
    push!(eqs, cos_coef ~ 0)
end

## Solve the nonlinear problem using ModelToolkit.jl and NonlinearSolve

In [ ]:
# Define the domain
domains = [x ∈ (xleft, xright)]  # Adjust domain as needed


# Create the PDESystem (it's actually an ODE system since there's only x)
Model.@named pde_system = Model.PDESystem(eqs, bcs, domains, [x], vars);

# Discretization
using MethodOfLines, DomainSets, NonlinearSolve

dx = 0.01  # Spatial step size
discretization = MOLFiniteDifference([x => dx], approx_order=2);

# Convert to ODEProblem
prob = discretize(pde_system, discretization);

# Solve
sol = solve(prob, NewtonRaphson(), abstol=1e-8, reltol=1e-8);

Once the system of differential equations is solved, extract the necessary coefficients

In [ ]:
# extract solution
solution_coeffs = []
for i in 1:(2*harmonics)
    # Extract solution    
    push!(solution_coeffs, sol[vars[i]]);
end

## Plot the results

In [ ]:
using CairoMakie
using GLMakie

Nt = 5

dt = 0.01
xgrid = collect(range(start=0.0, stop=1.0, step=dx))

# Observable for u(x,t)
# Initialize observable with an array of zeros
u = Observable(zeros(length(xgrid)))

fig = Figure(resolution = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)
lines!(ax, xgrid, u)
display(fig)

@async begin
    for k in 1:Int(Nt/dt)
        t_discrete = k * dt
        
        # Reset and compute the full sum
        u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solution_coeffs[i] .* sin(j * omega * t_discrete)
            else
                u_new .+= solution_coeffs[i] .* cos(j * omega * t_discrete)
                j += 1
            end
        end
        
        # Single assignment triggers the update
        u[] = u_new
        ax.title = "t = $(round(t_discrete, digits=2))"
        sleep(1/10)
    end
end